# Project Code
## Beer Preferences Across Amercia

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.graph_objs as go
import plotly as py
import plotly.figure_factory as ff
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler


ModuleNotFoundError: No module named 'plotly'

In [ ]:
import numpy as np
import warnings
warnings.simplefilter("ignore")

In [ ]:
data = pd.read_csv("beer_reviews.csv")

In [ ]:
data.head(10)

In [ ]:
print(data.info())
print('-'*50)
print(data.isna().sum())

In [ ]:
data['brewery_name'].fillna('unknown', inplace=True)
data['beer_abv'].fillna(data['beer_abv'].mean(), inplace=True)

data = data.drop(["brewery_id"], axis=1)
data = data.drop(["review_time"], axis=1)
data = data.drop(["review_profilename"], axis=1)
data = data.drop(["beer_beerid"], axis=1)
print(data.isna().sum())

In [ ]:
Top_beer_ratings = data[['beer_name','review_overall']].groupby('beer_name').review_overall.agg('mean').sort_values(ascending = False)
Top_beer_ratings = Top_beer_ratings.iloc[0:20]
Top_beer_ratings.reset_index()

In [ ]:
Top_rating_beer_styles = data[['beer_style','review_overall']].groupby('beer_style').review_overall.agg('mean').sort_values(ascending = False).iloc[0:20]
Top_rating_beer_styles = pd.DataFrame(Top_rating_beer_styles)
Top_rating_beer_styles.reset_index()

In [ ]:
beers_looks_smell = data[['beer_name', 'review_aroma','review_appearance','review_taste']]
beer_looks_smell = beers_looks_smell.groupby('beer_name').aggregate(['max'])
beer_looks_smell = beers_looks_smell.sort_values(by = ['review_aroma','review_appearance','review_taste'],ascending = False)
beers_looks_smell.iloc[0:20]

In [ ]:
top_ratings_taste = data[['brewery_name','beer_name','review_overall','review_taste']]
top_ratings_taste = top_ratings_taste.groupby(['brewery_name','beer_name']).aggregate(['mean'])
top_ratings_taste.reset_index()
top_ratings_taste.iloc[0:20]

In [ ]:
data.loc[ data[ 'beer_abv' ] > 30 ] \
    .sort_values( by = [ 'beer_abv' ], ascending = False )[ [ 'brewery_name', 'beer_name', 'beer_abv' ] ]

In [ ]:
linear_model = LinearRegression()

linear_model.fit( X = data[ [ 'review_aroma', 'review_appearance', 'review_palate', 'review_taste' ] ], y = data[ 'review_overall' ] )
preds = linear_model.predict(data[ [ 'review_aroma', 'review_appearance', 'review_palate', 'review_taste' ] ] )
linear_model.coef_

In [ ]:
data = data[['brewery_name', 'beer_style', 'beer_name', 'beer_abv', 'review_appearance', 
             'review_aroma', 'review_palate', 'review_taste', 'review_overall']]
data = data.sort_values(by=['brewery_name', 'beer_style', 'beer_name' , 'beer_abv', 'review_overall'])
data = data.reset_index()
data = data.drop(["index"], axis=1)

In [ ]:
data['review_average'] = data.apply(lambda row: (row["review_overall"] + row["review_aroma"] + 
                                                 row["review_appearance"] + row["review_palate"] + 
                                                 row["review_taste"]) / 5, axis=1)

data = data.drop(data[(data["review_average"] < 1) | (data["review_average"] > 5)].index)

In [ ]:
# Calculate average rating by beer style
avg_rating_by_style = data.groupby('beer_style')['review_overall'].mean().reset_index()


# Sort by average rating in descending order
avg_rating_by_style = avg_rating_by_style.sort_values(by='review_overall', ascending=False)

# Create bar plot using hvplot
bar_plot = avg_rating_by_style.hvplot.bar(x='beer_style', y='review_overall', 
                                          title='Average Ratings by Beer Style', 
                                          xlabel='Beer Style', ylabel='Average Rating',
                                          height=600,width=900, rot=90, 
                                          tools=['hover'])

# Show the plot
bar_plot

In [ ]:
def visualization_function_1(name, condition, ax_left, ax_right):
    
    unique_names = data[name].unique()
    length = len(unique_names)
    rev_aver = np.zeros(length)
    rev_taste = np.zeros(length)
    count = np.zeros(length, dtype=np.int32)
    
    for _, row in data.iterrows():
        idx = np.where(unique_names == row[name])
        rev_aver[idx] += row["review_appearance"]
        rev_taste[idx] += row["review_taste"]
        count[idx] += 1
        
  
    for i in range(length):
        if count[i] > condition:
            rev_aver[i] /= count[i]
            rev_taste[i] /= count[i]
        else:
            rev_aver[i] = 1
            rev_taste[i] = 1
            
    zipped_left  = sorted(zip(unique_names, rev_aver), key=lambda x: x[1], reverse=True)
    names_left   = [zipped_left[i][0] for i in range(length)]
    sorted_score = [zipped_left[i][1] for i in range(length)]
    
    zipped_right = sorted(zip(unique_names, rev_taste), key=lambda x: x[1], reverse=True)
    names_right  = [zipped_right[i][0] for i in range(length)]
    sorted_count = [zipped_right[i][1] for i in range(length)]
    
    sns.barplot(x = sorted_score[:15], y = names_left[:15], ax=ax_left)
    ax_left.set_xlim(3, 5)
    ax_left.set_xlabel("review_appearance")
    # limit the length of names
    ax_left.set_yticklabels([i.get_text()[:17] + "..." if len(i.get_text()) > 17 else i.get_text() 
                             for i in ax_left.get_yticklabels()])
    
    sns.barplot(x = sorted_count[:15], y = names_right[:15], ax=ax_right).set_title(name)
    ax_right.set_xlabel("review_taste")
    # limit the length of names
    ax_right.set_yticklabels([i.get_text()[:17] + "..." if len(i.get_text()) > 17 else i.get_text() 
                              for i in ax_right.get_yticklabels()])

sns.set(rc={"axes.grid": True})
fig, axs = plt.subplots(ncols=2, nrows=1, figsize=[16, 24])
fig.subplots_adjust(wspace=0.4)

visualization_function_1("beer_style", 5, axs[0], axs[1])
# visualization_function_1("beer_style", 5, axs[1, 0], axs[1, 1])
# visualization_function_1("beer_name", 5, axs[2, 0], axs[2, 1])

axs[0].set_title("The best appearance Beer Style")
axs[1].set_title("The best taste Beer Style")

# axs[1, 0].set_title("The best styles of beer")
# axs[1, 1].set_title("The most popular styles of beer")

# axs[2, 0].set_title("The best quality beer")
# axs[2, 1].set_title("The most popular beer");

In [ ]:
# we look at the hist of some reviews

# data = data.drop(data[data["beer_abv"] > 20].index)

sns.set(rc={"axes.grid": True})
fig, axs = plt.subplots(nrows=2, figsize=[16, 12])
plt.setp(axs, xticks=range(0, 21, 2))

sns.distplot(data["review_aroma"], bins=50, ax=axs[0], kde=False, color="m")
sns.kdeplot(data["review_aroma"], ax=axs[1], color="m", label="Original distribution")
sns.kdeplot(data["review_aroma"], ax=axs[1], color="r", bw=0.3, shade=True, label="Smoothed distribution")

axs[0].set_xlim(0, 6)
axs[0].set_ylabel("Total number of drinks")

axs[1].set_xlim(0, 6)
axs[1].set_xlabel("review_aroma")
axs[1].set_ylabel("Distribution");

In [ ]:
# Use only unique beers and breweries for aggregate stats
abv = data[['beer_name','brewery_name','beer_abv']]\
    .drop_duplicates(['beer_name','brewery_name'])\
    .pivot_table(index="brewery_name", aggfunc=("count",'mean','median'))

# Adjust column names
abv.columns = abv.columns.to_series().str.join('_')

# Reset index
abv.reset_index(inplace=True)

# Rename column
abv.rename(columns={'beer_abv_count':'beer_count'}, inplace=True)

# Filter for breweries with at least 4 unique beers
abv_filtered_mean = abv.query('beer_count >= 4')\
    .sort_values('beer_abv_mean', ascending=False)\
    .head(20)\
    .sort_values('beer_abv_mean', ascending=True)

# Plot it
p1 = [go.Bar(x = abv_filtered_mean['beer_abv_mean'] / 100,
             y = abv_filtered_mean['brewery_name'],
             text=abv_filtered_mean['beer_abv_mean'].round(1),
             hoverinfo='x',
             textposition = 'inside',
             orientation = 'h',
             opacity=0.7, 
             marker=dict(
                color='rgb(1, 87, 155)'))]

# Pieces of Flair
layout = go.Layout(title='Top 20 Breweries by Highest Mean ABV',
                   xaxis=dict(title="Mean ABV",
                              tickformat='%',
                              hoverformat = '.2%'),
                   margin = dict(l = 230),
                   font=dict(family='Courier New, monospace',
                            color='dark gray'))

fig = go.Figure(data=p1, layout=layout)

# Plot it
py.offline.iplot(fig)

In [ ]:
factors = ["review_appearance","review_aroma","review_palate","review_taste", "review_overall", "review_average"]

# Create a correlation matrix 
corr_metrics = data[factors].corr()
corr_metrics.style.background_gradient(cmap='Reds')

In [ ]:
# Method 2 - Heatmap: Plotly
heat_map_plotly = ff.create_annotated_heatmap(
    z=corr_metrics.values,
    x=list(corr_metrics.columns),
    y=list(corr_metrics.index),
    annotation_text=corr_metrics.round(2).values,
    showscale=True,
    colorscale='Portland',
    font_colors=['white','white'])

layout = go.Layout(title="Correlation Heatmap",
                   margin = dict(l = 330))

fig = go.Figure(data=heat_map_plotly, layout=layout)
py.offline.iplot(fig)

In [ ]:
data1 = data[['review_aroma','review_overall']]
beer_measures = data1.values
X = StandardScaler().fit_transform(beer_measures)


In [ ]:
# Calculate k-means fit for all Ks from 1 to 20.
k = range(1, 20)
kmeans = [KMeans(n_clusters=i) for i in k]
score = [kmeans[i].fit(X).score(X) for i in range(len(kmeans))]

# Plot the k-means score (percent variance explained) for each K.
fig = plt.figure(figsize=(16, 6))
plt.plot(k, score, '-o')
plt.xlabel('K Clusters')
plt.ylabel('Score')
plt.title('Elbow Curve');

In [ ]:
# classification model 
# categorize beers into quality levels based on the overall rating
# use random forest classifier

# define target variable (quality levels) and features
target = "quality"
features = ["review_aroma", "review_appearance", "review_palate", "review_taste", "beer_abv"]

# create target variable based on overall rating
data["quality"] = pd.cut(data["review_overall"], bins=[0, 2.5, 3.5, 5], labels=["Low", "Medium", "High"])

# split data into training and testing sets
X = data[features]
y = data[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# create and train the Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

# make predictions with test set
y_pred = rf_classifier.predict(X_test)

# evaluate performance
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", classification_rep)

# visualize the decision tree
# get the first decision tree in the Random Forest
first_tree = rf_classifier.estimators_[0]

dot_data = tree.export_graphviz(first_tree, out_file=None, 
                                feature_names=features, class_names=rf_classifier.classes_,
                                filled=True, rounded=True, special_characters=True, max_depth=3, rotate=True)  
graph = graphviz.Source(dot_data)  
# save the tree visualization as PDF for viewing
graph.render('random_forest_tree')  

In [ ]:
# user recomendation model 
# recomends a brewery based on preferences

# use a different test/train for this model
from surprise.model_selection import train_test_split

# take the average of multiple reviews for the same user and item
data = data.groupby(['brewery_name', 'beer_name']).mean().reset_index()

# convert 'beer_abv' column to numeric values
data['beer_abv'] = pd.to_numeric(data['beer_abv'], errors='coerce')

# drop rows with missing 'beer_abv' values
data.dropna(subset=['beer_abv'], inplace=True)

# convert 'brewery_name' and 'beer_name' columns to numeric identifiers
data['brewery_id'] = pd.factorize(data['brewery_name'])[0]
data['beer_id'] = pd.factorize(data['beer_name'])[0]

# prepare data for Surprise library
reader = Reader(rating_scale=(1, 5))
beer_ratings = Dataset.load_from_df(data[['brewery_id', 'beer_id', 'review_overall']], reader)

# split the data into training and testing sets
trainset, testset = train_test_split(beer_ratings, test_size=0.2, random_state=42)

# create and train the user-based collaborative filtering model
sim_options = {'name': 'cosine', 'user_based': True, 'verbose': False}
model = KNNBasic(sim_options=sim_options)
model.fit(trainset)

# function to get top N brewery recommendations for a user
def get_top_n_recommendations(user_id, n=10):
    user_brewery_ratings = [(brewery, model.predict(user_id, brewery).est) for brewery in data['brewery_id'].unique()]
    user_brewery_ratings.sort(key=lambda x: x[1], reverse=True)
    top_n_breweries = [brewery[0] for brewery in user_brewery_ratings[:n]]
    top_brewery_names = data[data['brewery_id'].isin(top_n_breweries)]['brewery_name'].unique()
    return top_brewery_names

# example usage: get top 5 brewery recommendations for a user with user_id 0
user_id = 0
top_breweries = get_top_n_recommendations(user_id, n=5)
print(f"Top Brewery Recommendations for user {user_id}:")
for i, brewery in enumerate(top_breweries, 1):
    print(f"{i}. {brewery}")